# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [4]:
# import library
import pandas as pd
import numpy as np
import matplotlib as plt
import re
from sqlalchemy import create_engine
%matplotlib inline

import pprint as pp

# import ml library
import nltk
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer

from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score,f1_score,recall_score
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer

# import pri library
import wrangling3 as w

# set env
## show all row and column
from IPython.display import display
#pd.set_option('display.max_rows', None)
#pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 200)
pd.set_option('display.max_columns', 200)
## show 200 string in context
pd.set_option('max_colwidth',200)

### // def load_data

In [9]:
# fac
def load_data(database_filepath):
    engine_name = 'sqlite:///' + database_filepath
    engine = create_engine(engine_name)
    df = pd.read_sql_table('YourTableName',con=engine)
    X = df['message']
    y = df[df.columns[5:]]
    
    return X,y

In [10]:
# fac
X,y = load_data('disastersql.db')

In [11]:
# fac
X.head(1)

0    Weather update - a cold front from Cuba that could pass over Haiti
Name: message, dtype: object

In [12]:
# fac
y.head(1)

request offer aid_related medical_help medical_products search_and_rescue  \
0       0     0           0            0                0                 0   

  security military child_alone water food shelter clothing money  \
0        0        0           0     0    0       0        0     0   

  missing_people refugees death other_aid infrastructure_related transport  \
0              0        0     0         0                      0         0   

  buildings electricity tools hospitals shops aid_centers  \
0         0           0     0         0     0           0   

  other_infrastructure weather_related floods storm fire earthquake cold  \
0                    0               0      0     0    0          0    0   

  other_weather direct_report  
0             0             0

In [14]:
def load_data():
    # load data from database
    engine = create_engine('sqlite:///InsertDatabaseName.db')
    df = pd.read_sql_table('InsertTableName',con=engine)
    X = df['message']
    y = df[df.columns[5:]]
    
    return X,y

In [15]:
# most columns are read as str
#df.info()

In [16]:
# genre、related are not popper for analysisi
#df.genre.value_counts()

In [24]:
X,y = load_data()

In [27]:
X.sample()

3515    I am in Turtle (Tortuga) Island, I did not find help and I am starving.. 
Name: message, dtype: object

In [28]:
Y.sample()

request offer aid_related medical_help medical_products  \
8508       1     0           1            0                1   

     search_and_rescue security military child_alone water food shelter  \
8508                 0        0        0           0     0    0       0   

     clothing money missing_people refugees death other_aid  \
8508        0     0              0        0     0         0   

     infrastructure_related transport buildings electricity tools hospitals  \
8508                      0         0         0           0     0         0   

     shops aid_centers other_infrastructure weather_related floods storm fire  \
8508     0           0                    0               0      0     0    0   

     earthquake cold other_weather direct_report  
8508          0    0             0             0

### 2. Write a tokenization function to process your text data

In [13]:
def tokenize(text):
    tokens = word_tokenize(text)
    lemmatizer = WordNetLemmatizer()
    
    clean_tokens = []
    for tok in tokens:
        clean_tok = lemmatizer.lemmatize(tok).lower().strip()
        clean_tokens.append(clean_tok)
    
    return clean_tokens

In [22]:
#nltk.download('wordnet')

### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [33]:
# shorter version
'''
pipeline = Pipeline([
        ('vect', CountVectorizer(tokenizer=tokenize)),
        ('tfidf', TfidfTransformer()),
        ('clf', RandomForestClassifier())
    ])
'''

In [34]:
# logger version
class StartingVerbExtractor(BaseEstimator, TransformerMixin):

    def starting_verb(self, text):
        sentence_list = nltk.sent_tokenize(text)
        for sentence in sentence_list:
            pos_tags = nltk.pos_tag(tokenize(sentence))
            first_word, first_tag = pos_tags[0]
            if first_tag in ['VB', 'VBP'] or first_word == 'RT':
                return True
        return False

    def fit(self, x, y=None):
        return self

    def transform(self, X):
        X_tagged = pd.Series(X).apply(self.starting_verb)
        return pd.DataFrame(X_tagged)

In [35]:
def model_pipeline():
    pipeline = Pipeline([
        ('features', FeatureUnion([

            ('text_pipeline', Pipeline([
                ('vect', CountVectorizer(tokenizer=tokenize)),
                ('tfidf', TfidfTransformer())
            ])),

            ('starting_verb', StartingVerbExtractor())
        ])),

        ('clf', RandomForestClassifier())
    ])

    return pipeline

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [36]:
y = y.astype(int)
y.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 26216 entries, 0 to 26215
Data columns (total 35 columns):
request                   26216 non-null int64
offer                     26216 non-null int64
aid_related               26216 non-null int64
medical_help              26216 non-null int64
medical_products          26216 non-null int64
search_and_rescue         26216 non-null int64
security                  26216 non-null int64
military                  26216 non-null int64
child_alone               26216 non-null int64
water                     26216 non-null int64
food                      26216 non-null int64
shelter                   26216 non-null int64
clothing                  26216 non-null int64
money                     26216 non-null int64
missing_people            26216 non-null int64
refugees                  26216 non-null int64
death                     26216 non-null int64
other_aid                 26216 non-null int64
infrastructure_related    26216 non-null int6

In [ ]:
#import wrangling3 as w
#w.checkvalue(y)

In [37]:
X_train, X_test, y_train, y_test = train_test_split(X, y)

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [38]:
model = model_pipeline()

In [39]:
model.fit(X_train, y_train);

/Users/francis/miniconda3/envs/py36/lib/python3.6/site-packages/sklearn/ensemble/forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


In [40]:
y_pred = model.predict(X_test)

In [63]:
accuracy_score(y_pred,y_test)

0.4171498321635642

In [61]:
f1_score(y_pred,y_test,average='weighted')

0.4875527250137354

In [62]:
recall_score(y_pred,y_test, average='weighted')

0.8103017926026775

In [78]:
target_names = df.columns[5:]
reports = classification_report(y_test,y_pred,target_names=target_names)

In [77]:
print(reports)

                        precision    recall  f1-score   support

               request       0.84      0.36      0.50      1084
                 offer       0.00      0.00      0.00        25
           aid_related       0.76      0.42      0.54      2676
          medical_help       0.50      0.01      0.02       525
      medical_products       0.79      0.03      0.07       323
     search_and_rescue       0.25      0.01      0.01       175
              security       0.50      0.01      0.02       125
              military       0.56      0.02      0.04       226
           child_alone       0.00      0.00      0.00         0
                 water       0.89      0.12      0.22       415
                  food       0.81      0.25      0.38       736
               shelter       0.89      0.14      0.24       565
              clothing       0.67      0.04      0.08        99
                 money       0.67      0.01      0.03       151
        missing_people       0.00      

In [82]:
# arc for other way to display result
'''
def display_results(y_test, y_pred):
    labels = np.unique(y_pred)
    #confusion_mat = confusion_matrix(y_test, y_pred, labels=labels)
    accuracy = (y_pred == y_test).mean()

    print("Labels:", labels)
    #print("Confusion Matrix:\n", confusion_mat)
    print("Accuracy:", accuracy)
''';

In [84]:
# archive (drop later)
# 可能和df读取的类型有关, 目前可以直接使用
# 如果不转换 y_pred 需要进行转换后比较
# print(classification_report(y_test.iloc[:,0], y_pred.T[0]))；

### 6. Improve your model
Use grid search to find better parameters. 

In [86]:
# slow lap, take small featrues to run grid
'''
parameters = {
        'features__text_pipeline__vect__ngram_range': ((1, 1), (1, 2)),
        'features__text_pipeline__vect__max_df': (0.5, 0.75, 1.0),
        'features__text_pipeline__vect__max_features': (None, 5000, 10000),
        'features__text_pipeline__tfidf__use_idf': (True, False),
        'clf__n_estimators': [50, 100, 200],
        'clf__min_samples_split': [2, 3, 4],
        'features__transformer_weights': (
            {'text_pipeline': 1, 'starting_verb': 0.5},
            {'text_pipeline': 0.5, 'starting_verb': 1},
            {'text_pipeline': 0.8, 'starting_verb': 1},
        )
    }
''';

In [111]:
def build_model():
    pipeline = Pipeline([
        ('vect', CountVectorizer(tokenizer=tokenize)),
        ('tfidf', TfidfTransformer()),
        ('clf', RandomForestClassifier())
    ])

    # specify parameters for grid search
    parameters = {
        'vect__max_df': (0.75, 1.0),
        'vect__max_features': (None, 5000),
        'tfidf__use_idf': (True, False),
        'clf__n_estimators': [50, 100],
        'clf__min_samples_split': [3, 4],
    }


    # create grid search object
    cv = GridSearchCV(pipeline, param_grid=parameters)
    
    #return pipeline
    return cv

In [97]:
def build_model():
    pipeline = Pipeline([
        ('features', FeatureUnion([
            
            ('text_pipeline', Pipeline([
                ('vect', CountVectorizer(tokenizer=tokenize)),
                ('tfidf', TfidfTransformer())
            ])),

            ('starting_verb', StartingVerbExtractor())
        ])),
    
        ('clf', RandomForestClassifier())
    ])

    # specify parameters for grid search
    parameters = {
        'features__text_pipeline__vect__max_df': (0.75, 1.0),
        'features__text_pipeline__vect__max_features': (None, 5000),
        'features__text_pipeline__tfidf__use_idf': (True, False),
        'clf__n_estimators': [50, 100],
        'clf__min_samples_split': [3, 4],
        'features__transformer_weights': (
            {'text_pipeline': 0.5, 'starting_verb': 1},
            {'text_pipeline': 0.8, 'starting_verb': 1},
        )
    }


    # create grid search object
    cv = GridSearchCV(pipeline, param_grid=parameters)
    
    return pipeline

In [113]:
# pipeline archive
'''
pipeline = Pipeline([
        ('vect', CountVectorizer(tokenizer=tokenize)),
        ('tfidf', TfidfTransformer()),
        ('clf', RandomForestClassifier())
    ])
    
    pipeline = Pipeline([
        ('features', FeatureUnion([

            ('text_pipeline', Pipeline([
                ('vect', CountVectorizer(tokenizer=tokenize)),
                ('tfidf', TfidfTransformer())
            ])),

            ('starting_verb', StartingVerbExtractor())
        ])),

        ('clf', RandomForestClassifier())
    ])   
'''

"\npipeline = Pipeline([\n        ('vect', CountVectorizer(tokenizer=tokenize)),\n        ('tfidf', TfidfTransformer()),\n        ('clf', RandomForestClassifier())\n    ])\n    \n    pipeline = Pipeline([\n        ('features', FeatureUnion([\n\n            ('text_pipeline', Pipeline([\n                ('vect', CountVectorizer(tokenizer=tokenize)),\n                ('tfidf', TfidfTransformer())\n            ])),\n\n            ('starting_verb', StartingVerbExtractor())\n        ])),\n\n        ('clf', RandomForestClassifier())\n    ])   \n"

In [114]:
# slow model (pass for laptop)
def build_model_slow():
    pipeline = Pipeline([
        ('features', FeatureUnion([
            
            ('text_pipeline', Pipeline([
                ('vect', CountVectorizer(tokenizer=tokenize)),
                ('tfidf', TfidfTransformer())
            ])),

            ('starting_verb', StartingVerbExtractor())
        ])),
    
        ('clf', RandomForestClassifier())
    ])

    # specify parameters for grid search
    parameters = {
        'features__text_pipeline__vect__max_df': (0.75, 1.0),
        'features__text_pipeline__vect__max_features': (5000,10000),
        'clf__n_estimators': [50, 100],
        'clf__min_samples_split': [3, 4],
        'features__transformer_weights': (
            {'text_pipeline': 0.5, 'starting_verb': 1},
            {'text_pipeline': 0.8, 'starting_verb': 1},
        )
    }


    # create grid search object
    cv = GridSearchCV(pipeline, param_grid=parameters)
    
    return cv

In [112]:
model = build_model()
model.fit(X_train, y_train)

/Users/francis/miniconda3/envs/py36/lib/python3.6/site-packages/sklearn/model_selection/_split.py:2053: FutureWarning: You should specify a value for 'cv' instead of relying on the default value. The default value will change from 3 to 5 in version 0.22.
  warnings.warn(CV_WARNING, FutureWarning)


GridSearchCV(cv='warn', error_score='raise-deprecating',
       estimator=Pipeline(memory=None,
     steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip...obs=None,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False))]),
       fit_params=None, iid='warn', n_jobs=None,
       param_grid={'vect__max_df': (0.75, 1.0), 'vect__max_features': (None, 5000), 'tfidf__use_idf': (True, False), 'clf__n_estimators': [50, 100], 'clf__min_samples_split': [3, 4]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, verbose=0)

### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

In [116]:
y_pred = model.predict(X_test)
reports = classification_report(y_test,y_pred,target_names=target_names)
print(reports)

# if enconter dtype('float64'), dtype('O'), it is because starting_verb fuc problem
# https://stackoverflow.com/questions/22273242/scipy-hstack-results-in-typeerror-no-supported-conversion-for-types-dtypef
# solve later

                        precision    recall  f1-score   support

               request       0.87      0.47      0.61      1084
                 offer       0.00      0.00      0.00        25
           aid_related       0.82      0.53      0.64      2676
          medical_help       0.50      0.01      0.02       525
      medical_products       0.78      0.02      0.04       323
     search_and_rescue       0.57      0.02      0.04       175
              security       0.50      0.01      0.02       125
              military       0.62      0.02      0.04       226
           child_alone       0.00      0.00      0.00         0
                 water       0.91      0.20      0.33       415
                  food       0.88      0.40      0.55       736
               shelter       0.92      0.22      0.36       565
              clothing       0.75      0.03      0.06        99
                 money       0.25      0.01      0.01       151
        missing_people       0.00      

### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

In [117]:
# to b c

### 9. Export your model as a pickle file

In [145]:
# export
import pickle
filename = 'p2_train_model.pickle'
pickle.dump(model, open(filename, 'wb'))
#pickle.dump(model, open(filename, 'wb'), protocol=True)
#https://blog.csdn.net/pipisorry/article/details/45315483
#测试了不同的protocol,区别不大

In [131]:
# shutill 需要指定目录才能处理单个文件
#https://stackoverflow.com/questions/30049201/how-to-compress-a-file-with-shutil-make-archive-in-python
'''
from shutil import make_archive

zipname = filename + '.zip'
make_archive(zipname,'zip',filename)
  
tarname = filename + '.tar.gz'
make_archive(tarname,'zip',filename)
''';

In [140]:
import tarfile
tar = tarfile.open(filename + ".tar.gz", mode='w|bz2')
tar.add("p2_train_model.pickle")
tar.close()
# 使用 tarbz 缩小为5%

In [126]:
# import
# original file is 900m, ziped 60M
# unzip before load
loaded_model = pickle.load(open(filename, 'rb'))
#https://machinelearningmastery.com/save-load-machine-learning-models-python-scikit-learn/

In [127]:
loaded_model

GridSearchCV(cv='warn', error_score='raise-deprecating',
       estimator=Pipeline(memory=None,
     steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip...obs=None,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False))]),
       fit_params=None, iid='warn', n_jobs=None,
       param_grid={'vect__max_df': (0.75, 1.0), 'vect__max_features': (None, 5000), 'tfidf__use_idf': (True, False), 'clf__n_estimators': [50, 100], 'clf__min_samples_split': [3, 4]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, verbose=0)

### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.